In [1]:
from collections import defaultdict
from dataclasses import dataclass
from pathlib import Path

from aoc.decorators import timeit

data_file = Path("../Data/day8.txt").read_text()

EXAMPLE = """..........
..........
..........
....a.....
..........
.....a....
..........
..........
..........
.........."""

EXAMPLE2 = """..........
..........
..........
....a.....
........a.
.....a....
..........
..........
..........
.........."""

EXAMPLE3 = """..........
..........
..........
....a.....
........a.
.....a....
..........
......A...
..........
.........."""

EXAMPLE4 = """............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""

EXAMPLE5 = """T.........
...T......
.T........
..........
..........
..........
..........
..........
..........
.........."""


@dataclass(frozen=True, eq=True)
class Point:
    x: int
    y: int

    @staticmethod
    def is_collinear(point_a: "Point", point_b: "Point", point_c: "Point") -> bool:
        cross_product = ((point_b.x - point_a.x) * (point_c.y - point_a.y)) - (
            (point_b.y - point_a.y) * (point_c.x - point_a.x)
        )

        return cross_product == 0


@dataclass
class RoofPlot:
    point: Point
    value: str


class RoofMap:
    grouped_antennas: defaultdict[str, list[RoofPlot]]

    __height: int
    __width: int

    def __init__(self, plots: list[list[RoofPlot]]) -> None:
        self.grouped_antennas = RoofMap.__get_antennas(plots)
        self.__height = len(plots) - 1
        self.__width = len(plots[0]) - 1

    @property
    def height(self):
        return self.__height

    @property
    def width(self):
        return self.__width

    def get_all_antennas(self):
        antenna_points: set[Point] = set()
        for antennas in self.grouped_antennas.values():
            for antenna in antennas:
                antenna_points.add(antenna.point)

        return antenna_points

    def get_all_anti_nodes(self, *, stride: bool = False):
        anti_nodes: set[Point] = set()
        for antennas in self.grouped_antennas.values():
            pairs = self.__make_pairs(antennas)
            anti_nodes.update(self.__get_anti_nodes(pairs, stride=stride))

        return anti_nodes

    @staticmethod
    def from_input(input: str):
        def map_line(enumerated_line: tuple[int, str]):
            x, line = enumerated_line

            return list(
                map(
                    lambda enumerated_value: RoofPlot(
                        point=Point(x=x, y=enumerated_value[0]),
                        value=enumerated_value[1],
                    ),
                    enumerate(line),
                )
            )

        return RoofMap(plots=list(map(map_line, enumerate(input.splitlines()))))

    def __get_anti_nodes(self, pairs: list[tuple[RoofPlot, RoofPlot]], *, stride: bool):
        anti_nodes: list[Point] = []
        for antenna_a, antenna_b in pairs:
            x_difference = abs(antenna_a.point.x - antenna_b.point.x)
            y_difference = abs(antenna_a.point.y - antenna_b.point.y)

            last_min_min = antenna_a.point
            last_min_plus = antenna_a.point
            last_plus_plus = antenna_b.point
            last_plus_min = antenna_b.point

            while (
                last_min_min is not None
                or last_min_plus is not None
                or last_plus_plus is not None
                or last_plus_min is not None
            ):
                last_min_min, last_min_plus, last_plus_plus, last_plus_min = (
                    self.__make_next_points_for_stride(
                        pair=(antenna_a, antenna_b),
                        x_difference=x_difference,
                        y_difference=y_difference,
                        min_min=last_min_min,
                        min_plus=last_min_plus,
                        plus_plus=last_plus_plus,
                        plus_min=last_plus_min,
                    )
                )
                if last_min_min:
                    anti_nodes.append(last_min_min)
                if last_min_plus:
                    anti_nodes.append(last_min_plus)
                if last_plus_plus:
                    anti_nodes.append(last_plus_plus)
                if last_plus_min:
                    anti_nodes.append(last_plus_min)

                if not stride:
                    break

        return anti_nodes

    def __make_next_points_for_stride(
        self,
        *,
        pair: tuple[RoofPlot, RoofPlot],
        x_difference: int,
        y_difference: int,
        min_min: Point | None,
        min_plus: Point | None,
        plus_plus: Point | None,
        plus_min: Point | None,
    ) -> tuple[
        Point | None,
        Point | None,
        Point | None,
        Point | None,
    ]:
        antenna_a, antenna_b = pair
        next_min_min: Point | None = None
        if min_min is not None:
            next_min_min = Point(
                x=min_min.x - x_difference,
                y=min_min.y - y_difference,
            )
            if not self.__is_within_bounds(next_min_min) or not Point.is_collinear(
                antenna_a.point, antenna_b.point, next_min_min
            ):
                next_min_min = None

        next_min_plus: Point | None = None
        if min_plus is not None:
            next_min_plus = Point(
                x=min_plus.x - x_difference,
                y=min_plus.y + y_difference,
            )
            if not self.__is_within_bounds(next_min_plus) or not Point.is_collinear(
                antenna_a.point, antenna_b.point, next_min_plus
            ):
                next_min_plus = None

        next_plus_plus: Point | None = None
        if plus_plus is not None:
            next_plus_plus = Point(
                x=plus_plus.x + x_difference,
                y=plus_plus.y + y_difference,
            )
            if not self.__is_within_bounds(next_plus_plus) or not Point.is_collinear(
                antenna_a.point, antenna_b.point, next_plus_plus
            ):
                next_plus_plus = None

        next_plus_min: Point | None = None
        if plus_min is not None:
            next_plus_min = Point(
                x=plus_min.x + x_difference,
                y=plus_min.y - y_difference,
            )
            if not self.__is_within_bounds(next_plus_min) or not Point.is_collinear(
                antenna_a.point, antenna_b.point, next_plus_min
            ):
                next_plus_min = None

        return next_min_min, next_min_plus, next_plus_plus, next_plus_min

    def __make_pairs(self, antennas: list[RoofPlot]) -> list[tuple[RoofPlot, RoofPlot]]:
        pairs: list[tuple[RoofPlot, RoofPlot]] = []
        for i, antenna_a in enumerate(antennas[:-1]):
            for antenna_b in antennas[i + 1 :]:
                pairs.append((antenna_a, antenna_b))

        return pairs

    def __is_within_bounds(self, point: Point):
        return (
            point.x >= 0
            and point.x <= self.width
            and point.y >= 0
            and point.y <= self.height
        )

    @staticmethod
    def __get_antennas(plots: list[list[RoofPlot]]) -> defaultdict[str, list[RoofPlot]]:
        antennas: defaultdict[str, list[RoofPlot]] = defaultdict(list)
        for row in plots:
            for plot in row:
                if plot.value != ".":
                    antennas[plot.value].append(plot)

        return antennas


def prepare(input: str):
    return RoofMap.from_input(input=input)

In [2]:
@timeit
def part1(input: str):
    roof_map = prepare(input)
    anti_nodes = roof_map.get_all_anti_nodes(stride=False)

    return len(anti_nodes)


example_result = part1(EXAMPLE)

assert (
    example_result == 2
), f"Expected example result to be 2, but got {example_result} instead"

example_result2 = part1(EXAMPLE2)

assert (
    example_result2 == 4
), f"Expected example 2 result to be 4, but got {example_result2} instead"

example_result3 = part1(EXAMPLE3)

assert (
    example_result3 == 4
), f"Expected example 3 result to be 4, but got {example_result3} instead"

example_result4 = part1(EXAMPLE4)

assert (
    example_result4 == 14
), f"Expected example 4 result to be 14, but got {example_result4} instead"

result = part1(data_file)

assert result < 330, f"Expected result to be less than 330, but got {result} instead"
assert result < 307, f"Expected result to be less than 307, but got {result} instead"
assert result > 285, f"Expected result to be greater than 285, but got {result} instead"
assert result == 303, f"Expected result to be 303, but got {result} instead"

print("result is", result)

def part1(input): took: 0.0001 sec
def part1(input): took: 0.0001 sec
def part1(input): took: 0.0001 sec
def part1(input): took: 0.0001 sec
def part1(input): took: 0.0020 sec
result is 303


In [3]:
@timeit
def part2(input: str):
    roof_map = prepare(input)
    anti_nodes = roof_map.get_all_anti_nodes(stride=True)
    antennas = roof_map.get_all_antennas()

    return len(antennas.union(anti_nodes))


example_result = part2(EXAMPLE4)

assert (
    example_result == 34
), f"Expected example result to be 34, but got {example_result} instead"


example_result2 = part2(EXAMPLE5)

assert (
    example_result2 == 9
), f"Expected example 2 result to be 9, but got {example_result2} instead"

result = part2(data_file)

print("result is", result)

assert result == 1045, f"Expected result to be 1045, but got {result} instead"

def part2(input): took: 0.0001 sec
def part2(input): took: 0.0001 sec
def part2(input): took: 0.0029 sec
result is 1045
